# Training data-efficient image transformers & distillation through attention  
- paper review

## Abstract  
- ViT는 수 억 장의 이미지로 사전 훈련을 해야한다는 단점 때문에 다양한 어플리케이션에 적용하는 데 한계가 있다  
- 본 논문에서 우리는 Convolution 없이 transformer로만 모델을 구축했으며 ImageNet 훈련만으로도 좋은 성능을 낸다는 것을 보여준다  
- 또한 transformer 학습에 특화된 teacher-student 학습 전략을 제안한다  
- DeiT는 CNN과 비교해 경쟁력 있는 성능을 보여준다  

## Introduction  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/figure1.png?raw=true" width=600>  

- vision 분야에서 attention mechanism을 활용한 Transformer 기반 모델 연구가 활발하다  
- 그 시도 중 하나가 CNN과 Transformer의 결합이다  
- ViT는 JFT-300M이라는 large scale dataset으로 사전 훈련을 시켜 transfer learning으로 CNN의 성능을 뛰어넘었다  
- ViT 논문에서 다음과 같은 결론을 내렸다  
- "Transforemr는 불충분한 양의 data로 훈련시키면 일반화가 잘 되지 않는다"  
- 그리고 많은 resource들을 필요로 한다  
- 반면 우리 모델은 single 8-GPU device로 2~3일 안에 학습을 시켰으며 CNN의 trainable parameter 수와 비슷하고 효율적이며 경쟁력 있는 성능을 보여준다  
- 우리는 Data-efficient Image Transformer (DeiT)를 제안하며 이전 모델보다 더 나은 결과를 보여준다  
- 또한, 우리는 token 기반 학습 전략을 제안하며 이는 일반적인 distillation보다 좋은 성능을 보여준다  
- contributions:  
    - 우리는 class token과 같은 역할을 하는 distillation token 기반 새로운 distillation proceduer를 제안한다  
    - 두 token은 attention을 통해 transformer에서 상호작용한다  
    - 이러한 distillation 학습 전략은 vanilla distillation보다 좋은 성능을 보여준다  
    - 흥미로운 점은, teacher model로 CNN을 사용한 것이 transformer를 사용한 것보다 학습이 더 잘 된다는 것이다  
    - DeiT를 ImageNet에서 사전 훈련을 한 후, CIFAR-10, CIFAR-100 등 다른 dataset으로 transfer learning을 하면 경쟁력 있는 성능을 보여준다  

## Related Work  
**Knowledge Distillation**  
- knowledge distillation method는 student model이 strong teacher network로부터 얻은 soft label을 활용하는 training paradigm을 의미한다  
- student model은 teacher model과 비슷한 성능을 내면서 더 적은 resource를 소모하기 위해 학습된다  
- 간단히 말해 student network가 teacher network의 성능을 모방할 수 있도록 하는 학습 전략이다  

## Vision transformer: overview  
**Multi-head Self Attention layers (MSA)**  
- query vector $q\in\mathcal{R}^d$는 내적을 이용해 key vector와 매칭된다  
- 그러면 내적된 값은 normalized한 후 softmax 연산을 거친 후 k weight를 얻을 수 있다  
$$Attention\left(Q,K,V\right)=Softmax\left(QK^T/\sqrt{d}\right)V$$  
- Query, Key, Value matrix는 N input vector를 입력으로 받으면 서로 연산된다 (self-supervised의 핵심)  
- 이때 $Q=XW_Q$, $K=W_K$, $V=XW_V$이며 $W_Q$, $W_K$, $W_V$는 모든 input vector 사이의 attention을 의미한다  
- 마지막을 Multi-head self-attention은 "heads"에 의해 정의된다  
- 각 head는 $N\times d$ size의 sequence를 제공한다  
- 이러한 $h$ sequence들은 linear layer에 의해 $N\times D$로 reprojection되는 $N\times dh$로 재배열된다  

**Transformer block for images**  
- 이미지를 처리하는 Transformer를 얻기 위해 우리 연구는 ViT model을 기반으로 한다  
- 고정된 사이즈의 RGB 이미지가 input으로 들어가면 $16\times 16$ 사이즈이 patch로 잘린다  
- 각 patch는 전체 dimension인 768을 보존하기 위해 lienar layer를 통해 projection된다  
- positional information은 고정되거나 학습 가능한 positional embedding을 포함한다  
- 이들은 first transformer block 전에 patch token에 더해지고 그런 다음, transformer block에 들어간다  

**The class token**  
- class token은 trainable vector이며 transformer에 들어가기 전에 patch token에 추가된다  
- 그러면 class를 예측하기 위해 linear layer에 의해 projection된다  
- 이러한 architecture는 patch token과 class token 사이에 information을 뿌리기 위해 self attention에 집중을 한다  

**Fixing the positional encoding across resolution**  
- "Fixing the train-test resolution discrepancy"라는 논문에서 학습할 때는 낮은 해상도로, fine-tuning 할 때는 더 큰 해상도로 하는 것이 바람직하다고 한다  
- 이러한 방법은 학습 속도를 높여주며 data augmentation을 사용할 때 정확도를 개선시켜준다  
- input 이미지의 해상도를 높이면 patch의 갯수가 늘어나게 된다  
- 이때 positional embedding은 patch마다 한 개씩 총 $N$개가 존재해 dimension을 맞춰야 한다  
- ViT 논문에서는 input 이미지의 해상도가 바뀌었을 때 positional encoding을 interpolation하며 이 method가 fine-tuning에서 작동함을 보여준다  

## Distillation through attention  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/figure2.png?raw=true" width=800>

- teacher model을 이용해 어떻게 transformer를 학습시킬 수 있는지 살펴본다  
- distillation method를 통해 transformer가 CNN을 대체할 수 있는지 살펴볼 것이다  
- 이 section에는 distillation에 대해 2가지를 살펴본다  
    - 1. hard distillation vs soft distillation  
    - 2. classicial distillation vs distillation token  
    
**Soft distillation**  
- 이는 Kullback-Leibler divergence를 사용하며 softmax가 적용된 teacher와 student의 output의 loss를 계산한다  
- 식은 다음과 같다  
$$\mathcal{L}_{global}=\left(1-\lambda\right)\mathcal{L}_{CE}\left(\psi\left(Z_s\right), y\right) + \lambda\tau^2KL\left(\psi\left(Z_s/\tau\right), \psi\left(Z_t/\tau\right)\right)$$  
- 이때 $Z_t$는 teacher network의 output, $Z_s$는 student network의 output  
- $\lambda$는 coefficient balancing parameter  
- $y$는 true label  
- $\psi$는 softmax function    

**Hard-label distillation**  
$$\mathcal{L}_{global}^{hardDistill}=\frac{1}{2}\mathcal{L}_{CE}\left(\psi\left(Z_s\right),y\right)+\frac{1}{2}\mathcal{L}_{CE}\left(\psi\left(Z_s\right), y_t\right)$$  
- hard label은 label smoothing 역할을 하는 soft label로 대체될 수 있다  
- smoothing label의 parameter $\epsilon$에 대해서 우리는 $\epsilon=0.1$로 고정을 시킨다  

**Distillation token**  
- 우리는 embedding에 distillation token을 추가한다  
- distillation token은 class token과 비슷한 역할을 한다  
- distillation embedding은 teacher의 output으로 나오며 학습되는 parameter이다  
- 흥미롭게도 distillation token은 처음에 학습할 때는 teacher의 label과 cosine 유사도가 0.06이었다  
- cosine 유사도가 0.93까지 도달하면서 1에 가까워지나, 1보다는 무조건 작다    

**Fine-tuning with distillation**  
- 우리는 더 높은 해상도로 fine-tuning stage에서 true label과 teacher의 prediction 둘 다 사용해봤다  
- 그 결과, true label은 teacher의 benefit을 감소시키며 성능 또한 낮아졌다  

## Experiments  
- 먼저 우리의 distillation method에 대해 살펴보고 CNN과 Transformer를 비교한다  

### Transformer models  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table1.png?raw=true" width=700>

- 우리 모델의 디자인은 ViT와 같다  
- 다른 점은 training 방식과 distillation token의 유무다  
- 또한 우리는 pre-training을 위한 MLP classifier를 이용하지 않았다  

### Distillation  
- 여러 실험을 한 결과, 흥미로운 것은 student model이 teacher model의 성능을 능가한다는 것이다  
- ImageNet-1K에서 우리의 best model은 85.1%의 정확도를 달성하며 JFT-300M으로 사전훈련한 ViT-B보다 성능이 좋다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table2.png?raw=true" width=700>

**Convnets teachers**  
- 우리는 teacher model로 transformer보다 CNN을 사용했을 때 더 성능이 좋다는 것을 관찰했다  
- table 2에서 이를 보여주고 있다  
- inductive bias 측면에서 확률적으로 CNN이 transformer보다 많은 것은 사실이다  
- 이제 우리는 distillation 실험에서 teacher model로 RegNetY-16GF를 사용할 것이다  
- 이때 DeiT와 같은 data로 실험하며 같은 augmentation을 적용한다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table3.png?raw=true" width=700>

**Comparison of distillation methods**  
- hard distillation이 soft distillation보다 성능이 뛰어나다  
- 우리의 distillation 전략은 성능을 개선시켰으며 class token까지 사용했을 때 가장 좋은 성능을 보여준다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table4.png?raw=true" width=700>

**Agreement with the teacher & inductive bias?**  
- distilled model은 transformer보다 CNN 모델과 더 상관이 있다  
- distillation embedding의 classifier는 class embedding + CNN에 더 유사하다  
- 반대로, class embedding classifier는 distillation 없이 학습된 DeiT와 유사하다  

**Number of epochs**  
- distillation을 이용해 학습시킬 때는 성능이 향상되는 모습을 보여준다  
- 300 epoch에서 DeiT-B보다 distillation을 사용한 DeiT-B 모델이 더 성능이 좋다  
- distillated network의 경우 오래 학습할수록 좋은 성능을 얻는다  

### Efficiency vs accuracy: a comparative study with convnets  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table5.png?raw=true" width=700>

- 우리는 throughput과 accuracy 사이의 trade-off에 집중했다  
- DeiT는 EfficientNet보다 성능이 약간 낮다  
- 하지만 CNN과 Transformer 사이의 성능 격차를 좁혔다는 유의마한 결과를 얻었다  
- 또한 기존ViT보다 6.3% 더 좋은 성능을 얻었다  
- 그리고 DeiT가 RegNetY를 이용해 distillation 학습을 시키면 EfficientNet보다 조금 더 좋은 성능을 얻는다  

### Transfer learning: Performance on downstream tasks  
- 우리는 각 dataset에 대해 fine-tuning을 이용해 transfer learning을 적용해 성능을 체크해봤다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table6.png?raw=true" width=500>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table7.png?raw=true" width=700>  

- 기존 ViT보다 더 좋은 성능을 보여주며 SOTA CNN기반 모델과 비교해도 경쟁력 있는 성능을 가진다  

## Training details & ablation  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DeiT/figure/table9.png?raw=true" width=700>

**Initialization and hyper-parameters**  
- table 9에서 실험에 사용한 hyperparameter들의 default 값을 나타냈다  
- distillation 학습에 대해선 $\tau=3$, $\lambda=0.1$의 값을 사용했다  

**Data-Augmentation**  
- Transformer는 많은 data를 필요로 한다  
- 따라서 우리는 data augmentation을 활용한다  
- 최대한 많은 augmentation을 활용해 성능 개선을 얻었으며 dropout을 활용하지 않았다  

**Regularization & Optimizers**  
- Transformer는 hyperparameter setting에 민감하다  
- 따라서 3개의 learning rate $\left(5\cdot 10^{-4}, 3\cdot 10^{-4}, 5\cdot 10^{-5}\right)$와 3개의 weight decay $\left(0.03, 0.04, 0.05\right)$의 값으로 실험을 한다  
- 우리는 lr을 조절하는데 다음과 같은 식을 이용한다  
$$lr_{scaled}=\frac{lr}{512}\times batch size$$  
- AdamW optimizer와 같은 lr을 사용했을 때 가장 좋은 결과를 얻었다  
- Mixup이나 CutMix 같은 Regularization도 성능을 개선시켜줬으며 repeated augmentation을 이용했다  

**Fine-tuning at different resolution**  
- 우리는 positional encoding을 interpolation을 한다  
- 주로 bilinear method를 사용한다  
- 그러나 주변 vector의 bilinear interpolation은 l2-norm을 감소시킨다는 점이 있다  
- 이러한 low-norm은 pre-trained Transformer에 적용되지 않으며 fine-tuning 없이 바로 사용할 경우 정확도가 크게 떨어진다  
- 따라서 우리는 bicubic interpolation을 채택하며 AdamW 또는 SGD로 fine-tuning하기 전에 적용한다  

**Training time**  
- DeiT-B 모델을 300 epoch으로 학습할 경우, 2 nodes GPU는 37시간, 1 node GPU는 53시간이 걸렸다  
- DeiT-S와 Tiny model은 4 GPU로 학습시킬 때 3일도 걸리지 않았다  

## Conclusion  
- 본 논문에서 우리는 DeiT를 소개했으며 large scale dataset으로 사전훈련할 필요가 없고 특히 새로운 distillation 전략으로 인해 더 좋은 성능을 가진다  
- 우리는 augmentation과 regularization에 집중을 했으며 distillation token을 추가하는 것 외에 architecture의 수정이 없었다  
- 따라서 Transformer에 적합한 augmentation 등에 대한 연구를 통해 더 많은 이점을 가져올 수 있다  
- 더 적은 resource를 고려한다면 우리 DeiT가 좋은 옵션이 될 것이다